In [1]:
import os
import sys
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)
    
import matplotlib.pyplot as plt
from src.config import get_dataset_and_user
%matplotlib inline

In [2]:
# get data
X_iris, user_iris = get_dataset_and_user('iris')
y_iris = user_iris.get_label(X_iris, update_counter=False)

X_housing, user_housing = get_dataset_and_user('housing')
y_housing = user_housing.get_label(X_housing, update_counter=False)

In [3]:
from sklearn.svm import LinearSVC, SVC
from sklearn.metrics import f1_score
clf = SVC(C=1000, kernel='rbf')
clf.fit(X_iris,y_iris)
f1_score(clf.predict(X_iris), y_iris)

1.0

In [4]:
from src.active_learning.svm import SimpleMargin, SolverMethod, OptimalMargin
from src.active_learning.linear import LinearMajorityVote
from src.active_learning.agnostic import RandomLearner
from src.showdown import Showdown
from src.plotting import plot_showdown

user_housing.max_iter = 50
user_iris.max_iter = 10

active_learners_list = [
    ("random", RandomLearner(SVC(C=100, kernel='rbf'))),
    #("optimal linear", OptimalMargin(C=100, kind='kernel', kernel='rbf', chain_length=50, sample_size=8, threshold=0.9, factorization_type='diagonalize')),
    ("simple margin", SimpleMargin(kind='kernel', kernel='rbf', C=100)),
    #('cholesky CL=50', OptimalMargin(C=100, kind='kernel', kernel='rbf', chain_length=50, factorization_type='cholesky')),
    #('diag th=0.9 CL=50', OptimalMargin(C=100, kind='kernel', kernel='rbf', chain_length=50, threshold=0.9, factorization_type='diagonalize')),
    #('diag th=0.9 CL=50 SS=8', OptimalMargin(C=100, kind='kernel', kernel='rbf', chain_length=50, sample_size=8, threshold=0.9, factorization_type='diagonalize')),
    #('diag th=0.9 CL=50, SS=16', OptimalMargin(C=100, kind='kernel', kernel='rbf', chain_length=50, sample_size=16, threshold=0.9, factorization_type='diagonalize')),
]

datasets_list = [
    ("housing", X_housing, user_housing),
    ("iris", X_iris, user_iris),
]

times = 20
showdown = Showdown()
output = showdown.run(datasets_list, active_learners_list, times)

#print(output.head())
plot_showdown(output, times, metrics_list=['fscore', 'iteration_time'])

TypeError: zip argument #2 must support iteration

## Next steps
    - add logging
    - label points prematurely based on whether they cut the current version space or not (cut estimate = # samples)
    - add offset (add column of 1's ?) -> works on above cases. Should test on sdss also.
    - change classifier to average of sampled w's (similar to majority vote in ALuMa)
    - add sdss queries
    - initial sampling: use sampling to pick k points which closely cut the VS in 1/2^k proportion / probability of each region close to uniform (entropy criteria?). Can we avoid enumerating all k subsets?
    - start using majoirity vote until 1 pos and 1 neg point have been found?
    
    
    - dozens of labeled samples, low selectivity
    - Problems: categorical attibutes, few positive samples, noise 
    - try queries from sdss paper (user rbf kernel) -> see if these algorithms work better than simple margin 

In [ ]:
from sklearn.metrics.pairwise import rbf_kernel
import numpy as np
K = rbf_kernel(X_housing.values)
lamb, P = np.linalg.eigh(K + 1e-8 * np.eye(len(X_housing)))
L = P.dot(np.sqrt(np.diag(lamb)))

In [ ]:
output['housing']['fscore'].swaplevel(0,1, axis=1).sort_index(level=0, axis=1)